In [16]:
import pandas as pd
import numpy as np
import re
import requests
import bs4
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common.exceptions import *
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.alert import Alert
import time

# Initiation 
##### Lists for Storing Infos

In [17]:
#토지정보
address_lst = [] #지번주소
category_lst = [] #지목
area_lst = [] #면적
scale_lst = [] #축척
own_lst = [] #소유
building_lst = [] #건물명
ownchg_ymd_lst = [] #소유권 변동일자
ownchg_rsn_lst = [] #소유권 변동원인
move_ymd_lst = [] #토지이동일자
move_rsn_lst = [] #토지이동사유

#토지표시연혁
landmark_history_address = [] #지번주소
landmark_cate_lst = [] #지목
landmark_area_lst = [] #면적
landmark_move_rsn_lst = [] #토지이동사유
landmark_move_ymd_lst = [] #토지이동일자
landmark_related_address_lst = [] #관련지번

#고유번호 변동연혁
id_history_address = [] #지번주소
bf_location_lst = [] #이전대지위치
num_move_rsn_lst = [] #토지이동사유
num_move_ymd_lst = [] #토지이동일자

#토지등급 연혁
grade_history_address = [] #지번주소
grade_lst = [] #등급
grade_cate_lst = [] #등급구분
gradecng_ymd_lst = [] #등급변동일자
gradecng_cate_lst = [] #등급변동구분
    
#개별 공시지가
indv_value_address = [] #지번주소
disclosure_ym_lst = [] #공시기준일
indv_value_lst = [] #개별공시지가

# Define Function
##### 지번수를 매개변수로 함
### 1) 토지 기본정보

In [18]:
def get_info(sp): #sp는 검색결과에 뜨는 ~번째 주소
    driver.implicitly_wait(10)
    
    #토지정보 클릭
    click('#lotTab02')
    driver.implicitly_wait(10)
    
    #종합정보 xpath -> text
    address = xpath_to_text('//*[@id="lotJibunNm"]') #지번주소
    driver.implicitly_wait(10)

    #토지정보 xpath -> text
    category = xpath_to_text('//*[@id="ladJimk"]') #지목
    area = xpath_to_text('//*[@id="ladArea"]') #면적
    scale = xpath_to_text('//*[@id="ladScale"]') #축척
    own = xpath_to_text('//*[@id="ladOwnGbn"]') #소유
    building = xpath_to_text('//*[@id="ladBuldNm"]') #건물명
    ownchg_ymd = xpath_to_text('//*[@id="ladOwnChgYmd"]') #소유권 변동일자
    ownchg_rsn = xpath_to_text('//*[@id="ladOwnChgRsn"]') #소유권 변동원인
    move_ymd = xpath_to_text('//*[@id="ladMoveYmd"]') #토지이동일자
    move_rsn = xpath_to_text('//*[@id="ladMoveWhy"]') #토지이동사유
    driver.implicitly_wait(10)
    
    #기본 토지정보 리스트에 추가
    address_lst.append(address)
    category_lst.append(category)
    area_lst.append(area)
    scale_lst.append(scale)
    own_lst.append(own)
    building_lst.append(building)
    ownchg_ymd_lst.append(ownchg_ymd)
    ownchg_rsn_lst.append(ownchg_rsn)
    move_ymd_lst.append(move_ymd)
    move_rsn_lst.append(move_rsn)


### 2) 토지표시연혁

In [19]:
def get_info_landmark(sp): 
    address = xpath_to_text('//*[@id="lotJibunNm"]') #지번주소
    driver.implicitly_wait(10)
    
    num = 1
    while True:
        try:
            if xpath_to_text('//*[@id="ladMovHistTb"]/tr/td') == '조회된 결과가 없습니다.':
                driver.implicitly_wait(10)
                break
                
            else:
                driver.implicitly_wait(10)
                landmark_cate = xpath_to_text(f'//*[@id="ladMovHistTb"]/tr[{num}]/td[1]')
                driver.implicitly_wait(10)
                landmark_cate_lst.append(landmark_cate)
                driver.implicitly_wait(10)
                landmark_area = xpath_to_text(f'//*[@id="ladMovHistTb"]/tr[{num}]/td[2]')
                driver.implicitly_wait(10)
                landmark_area_lst.append(landmark_area)
                driver.implicitly_wait(10)
                landmark_move_rsn = xpath_to_text(f'//*[@id="ladMovHistTb"]/tr[{num}]/td[3]')
                driver.implicitly_wait(10)
                landmark_move_rsn_lst.append(landmark_move_rsn)
                driver.implicitly_wait(10)
                landmark_move_ymd = xpath_to_text(f'//*[@id="ladMovHistTb"]/tr[{num}]/td[4]')
                driver.implicitly_wait(10)
                landmark_move_ymd_lst.append(landmark_move_ymd)
                driver.implicitly_wait(10)
                landmark_related_address = xpath_to_text(f'//*[@id="ladMovHistTb"]/tr[{num}]/td[5]')
                driver.implicitly_wait(10)
                landmark_related_address_lst.append(landmark_related_address)
                driver.implicitly_wait(10)
                landmark_history_address.append(address)
                driver.implicitly_wait(10)
                num += 1
                driver.implicitly_wait(10)
                time.sleep(3)
                
        except NoSuchElementException:
            break
        except StaleElementReferenceException:
            continue

### 3) 고유번호 변동연혁

In [20]:
def get_info_id(sp):
    
    address = xpath_to_text('//*[@id="lotJibunNm"]') #지번주소
    driver.implicitly_wait(10)
    
    #고유번호 변동연혁
    click('#ui-cont-eachlot > div.cont-wrap > div.cont-tab-pack > ul.cont-tab-box > li.on > div > div.cont-tab-inner-pack > ul.cont-tab-inner.unit5 > li:nth-child(2) > a > span')
    driver.implicitly_wait(10)
    
    num = 1
    while True:
        try:
            if xpath_to_text('//*[@id="ladPnuChgHistTb"]/tr/td') == '조회된 결과가 없습니다.':
                driver.implicitly_wait(10)
                break
                
            else:
                driver.implicitly_wait(10)
                bf_location = xpath_to_text(f'//*[@id="ladPnuChgHistTb"]/tr[{num}]/td[1]')
                driver.implicitly_wait(10)
                bf_location_lst.append(bf_location)
                driver.implicitly_wait(10)
                num_move_rsn = xpath_to_text(f'//*[@id="ladPnuChgHistTb"]/tr[{num}]/td[2]')
                driver.implicitly_wait(10)
                num_move_rsn_lst.append(num_move_rsn)
                driver.implicitly_wait(10)
                run_move_ymd = xpath_to_text(f'//*[@id="ladPnuChgHistTb"]/tr[{num}]/td[3]')
                driver.implicitly_wait(10)
                num_move_ymd_lst.append(run_move_ymd)
                driver.implicitly_wait(10)
                id_history_address.append(address)
                driver.implicitly_wait(10)
                num += 1
                driver.implicitly_wait(10)
                time.sleep(3)
                
        except NoSuchElementException:
            break
        except StaleElementReferenceException:
            continue

### 4) 토지등급 연혁

In [21]:
def get_info_grade(sp):

    address = xpath_to_text('//*[@id="lotJibunNm"]') #지번주소
    driver.implicitly_wait(10)
    
    #토지등급 연혁
    click('#ui-cont-eachlot > div.cont-wrap > div.cont-tab-pack > ul.cont-tab-box > li.on > div > div.cont-tab-inner-pack > ul.cont-tab-inner.unit5 > li:nth-child(3) > a > span')
    driver.implicitly_wait(10)
    
    num = 1
    while True:
        try:
            if xpath_to_text('//*[@id="ladRnkHistTb"]/tr/td') == '조회된 결과가 없습니다.':
                break
                
            else:
                driver.implicitly_wait(10)
                grade = xpath_to_text(f'//*[@id="ladRnkHistTb"]/tr[{num}]/td[1]')
                driver.implicitly_wait(10)
                grade_lst.append(grade)
                driver.implicitly_wait(10)
                grade_cate = xpath_to_text(f'//*[@id="ladRnkHistTb"]/tr[{num}]/td[2]')
                driver.implicitly_wait(10)
                grade_cate_lst.append(grade_cate)
                driver.implicitly_wait(10)
                gradecng_ymd = xpath_to_text(f'//*[@id="ladRnkHistTb"]/tr[{num}]/td[3]')
                driver.implicitly_wait(10)
                gradecng_ymd_lst.append(gradecng_ymd)
                driver.implicitly_wait(10)
                gradecng_cate = xpath_to_text(f'//*[@id="ladRnkHistTb"]/tr[{num}]/td[4]')
                driver.implicitly_wait(10)
                gradecng_cate_lst.append(gradecng_cate)
                driver.implicitly_wait(10)
                grade_history_address.append(address)
                driver.implicitly_wait(10)
                num += 1
                driver.implicitly_wait(10)
                time.sleep(3)
                
        except NoSuchElementException:
            break
        except StaleElementReferenceException:
            continue

### 5) 개별 공시지가

In [22]:
def get_info_indv(sp):

    address = xpath_to_text('//*[@id="lotJibunNm"]') #지번주소
    driver.implicitly_wait(10)
    
    #개별 공시지가
    click('#ui-cont-eachlot > div.cont-wrap > div.cont-tab-pack > ul.cont-tab-box > li.on > div > div.cont-tab-inner-pack > ul.cont-tab-inner.unit5 > li:nth-child(5) > a > span')
    driver.implicitly_wait(10)
    
    num = 1
    while True:
        try:
            if xpath_to_text('//*[@id="ladPriceTb"]/tr/td') == '조회된 결과가 없습니다.':
                driver.implicitly_wait(10)
                break
                
            else:
                driver.implicitly_wait(10)
                disclosure_ym = xpath_to_text(f'//*[@id="ladPriceTb"]/tr[{num}]/td[1]')
                driver.implicitly_wait(10)
                disclosure_ym_lst.append(disclosure_ym)
                driver.implicitly_wait(10)
                indv_value = xpath_to_text(f'//*[@id="ladPriceTb"]/tr[{num}]/td[2]')
                driver.implicitly_wait(10)
                indv_value_lst.append(indv_value)
                driver.implicitly_wait(10)
                indv_value_address.append(address)
                driver.implicitly_wait(10)
                num += 1
                driver.implicitly_wait(10)
                time.sleep(3)
                
        except NoSuchElementException:
            break
        except StaleElementReferenceException:
            continue


# Selenium Driver Crawling

### Function "info_city": 시/군/구 번호 & 동 번호 입력
##### 시/군/구 번호

2: 가평군, 
3: 고양시덕양구, 
4: 고양시일산동구, 
5: 고양시일산서구. 
6: 과천시
<br>
7: 광명시, 
8: 광주시, 
9: 구리시, 
10: 군포시. 
11: 김포시
<br>
12: 남양주시, 
13: 동두천시, 
14: 부천시, 
15: 성남시분당구, 
16: 성남시수정구
<br>
17: 성남시중원구, 
18: 수원시권선구, 
19: 수원시영통구, 
20: 수원시장안구, 
21: 수원시팔달구
<br>
22: 시흥시, 
23: 안산시단원구, 
24: 안산시상록구, 
25: 안성시, 
26: 안양시동안구
<br>
27: 안양시만안구, 
28: 양주시, 
29: 양평군, 
30: 여주시, 
31: 연천군
<br>
32: 오산시, 
33: 용인시기흥구, 
34: 용인시수지구, 
35: 용인시처인구, 
36: 의왕시
<br>
37: 의정부시, 
38: 이천시, 
39: 파주시, 
40: 평택시, 
41: 포천시
<br>
42: 하남시, 
43: 화성시

In [28]:
def info_city(city, dong):
    
    #'시' 설정
    click(f'#selSigungu > option:nth-child({city})')

    #'동' 설정
    click(f'#selUmdRi > option:nth-child({dong})')

    for num in np.arange(1, 20): #999개에서 9999개로 변경함
        sp = 1
        
        driver.implicitly_wait(10)
        address_num = driver.find_element_by_css_selector('#bonbnText')
        address_num.clear()
        address_num.send_keys(str(num)) #지번입력

        #검색버튼 클릭
        click('#jibunForm > input.submit')
        driver.implicitly_wait(10)
        
        while True: 
            try:
                #주소 검색결과 sp 번째 결과 클릭
                driver.implicitly_wait(10)
                click(f'#addrTable > li:nth-child({sp}) > a')
                driver.implicitly_wait(10)
                get_info(str(sp))
                driver.implicitly_wait(10)
                get_info_landmark(str(sp))
                driver.implicitly_wait(10)
                get_info_id(str(sp))
                driver.implicitly_wait(10)
                get_info_grade(str(sp))
                driver.implicitly_wait(10)
                get_info_indv(str(sp))
                driver.implicitly_wait(10)
                sp += 1
                driver.implicitly_wait(10)
                click('#ui-cont-search-id > span')
                driver.implicitly_wait(10)
                
            except NoSuchElementException:
                break
            except StaleElementReferenceException:
                continue
            except UnexpectedAlertPresentException: #위치정보 부재로 인한 경고알림창
                try:
                    da = Alert(driver)
                    da.accept()
                    continue
                except NoAlertPresentException:
                    driver.implicitly_wait(10)
                    get_info(str(sp))
                    driver.implicitly_wait(10)
                    get_info_landmark(str(sp))
                    driver.implicitly_wait(10)
                    get_info_id(str(sp))
                    driver.implicitly_wait(10)
                    get_info_grade(str(sp))
                    driver.implicitly_wait(10)
                    get_info_indv(str(sp))
                    driver.implicitly_wait(10)
                    sp += 1
                    driver.implicitly_wait(10)
                    click('#ui-cont-search-id > span')
                    continue
                    
            except ElementNotInteractableException:   
                continue
            except ElementClickInterceptedException:
                continue


In [24]:
#selector click function
def click(selector):
    button = driver.find_element_by_css_selector(selector)
    driver.implicitly_wait(10)
    button.click()

#xpath to text transform function
def xpath_to_text(path):
    text = driver.find_element_by_xpath(path).text
    driver.implicitly_wait(10)
    return text 

================================================================================================================================
* 위 셀까지 무조건 실행 후
* 아래 세 옵션 중 선택하여 실행

### 특정 동(dong)에 대한 작업수행

In [29]:
driver = webdriver.Chrome(r'C:\Users\esthe\chromedriver.exe')
site = 'https://gris.gg.go.kr/map/main/grisMapView.do'
driver.get(site) 

#'검색창' 선택
click('#ui-cont-search-id > span')

#'지번주소로 검색' 선택
click('#ui-cont-search > div.ui-search > div > a:nth-child(2)')

info_city(43, 4) #화성시 금곡동

### 특정 시(city)에 대한 작업수행

In [21]:
driver = webdriver.Chrome(r'C:\Users\esthe\chromedriver.exe')
site = 'https://gris.gg.go.kr/map/main/grisMapView.do'
driver.get(site) 

#'검색창' 선택
click('#ui-cont-search-id > span')

#'지번주소로 검색' 선택
click('#ui-cont-search > div.ui-search > div > a:nth-child(2)')

dong = 2 #첫 동 > 2로 시작됨
while True:
    try:
        info_city(43, dong) #화성시로 설정
        driver.implicitly_wait(10)
        dong += 1
        driver.implicitly_wait(10)
    except NoSuchElementException:
        break
    except ElementNotInteractableException:
        click('#ui-cont-search-id > span')
        driver.implicitly_wait(10)
        continue

### 경기도 전체에 대한 작업수행

In [17]:
driver = webdriver.Chrome(r'C:\Users\esthe\chromedriver.exe')
site = 'https://gris.gg.go.kr/map/main/grisMapView.do'
driver.get(site) 

#'검색창' 선택
click('#ui-cont-search-id > span')

#'지번주소로 검색' 선택
click('#ui-cont-search > div.ui-search > div > a:nth-child(2)')

for city in np.arange(2, 44):
    dong = 2
    driver.implicitly_wait(10)
    
    while True:
        try:
            info_city(city, dong) #화성시
            driver.implicitly_wait(10)
            dong += 1
            driver.implicitly_wait(10)
        except NoSuchElementException:
            break
        except ElementNotInteractableException:
            click('#ui-cont-search-id > span')
            driver.implicitly_wait(10)
            continue

# DataFrame_to_CSV

In [33]:
data_general = {'address' : address_lst,
          'category' : category_lst, 
          'area' : area_lst, 
          'scale' : scale_lst,
          'own' : own_lst, 
          'building' : building_lst, 
          'ownchg_ymd' : ownchg_ymd_lst, 
          'ownchg_rsn' : ownchg_rsn_lst,
          'move_ymd' : move_ymd_lst,
           'move_rsn' : move_rsn_lst}

data_land_history = {'address' : landmark_history_address, 
                 'category' : landmark_cate_lst, 
                 'area' : landmark_area_lst, 
                 'move_rsn' : landmark_move_rsn_lst, 
                 'move_ymd' : landmark_move_ymd_lst, 
                 'related_address' : landmark_related_address_lst}

data_id_history = {'address' : id_history_address, 
               'bf_location' : bf_location_lst, 
               'num_move_rsn' : num_move_rsn_lst, 
               'num_move_ymd' : num_move_ymd_lst}

data_grade_history = {'address' : grade_history_address, 
               'grade' : grade_lst, 
               'grade_category' : grade_cate_lst, 
               'grade_cange_ymd' : gradecng_ymd_lst, 
               'grade_change_category' : gradecng_cate_lst}

data_indv_value_history = {'address' : indv_value_address, 
               'disclosure_ym' : disclosure_ym_lst, 
               'indv_value' : indv_value_lst}

#데이터프레임 생성
df = pd.DataFrame(data_general)
#df2 = pd.DataFrame(data_land_history)
df3 = pd.DataFrame(data_id_history)
df4 = pd.DataFrame(data_grade_history)
df5 = pd.DataFrame(data_indv_value_history)

In [156]:
#csv 파일 저장 (경로지정 필요함)
df.to_csv('C:\RA\general.csv', index = False)
df2.to_csv('C:\RA\landmark.csv', index = False)
df3.to_csv('C:\RA\id.csv', index = False)
df4.to_csv('C:\RA\grade.csv', index = False)
df5.to_csv('C:\RA\indv.csv', index = False)

In [36]:
df4

,address,grade,grade_category,grade_cange_ymd,grade_change_category
0,화성시 기산동 1-1,119,토지등급,1986-08-01,수정
1,화성시 기산동 1-1,117,토지등급,1985-07-01,수정
2,화성시 기산동 1-1,98,토지등급,1984-07-01,수정
3,화성시 기산동 1-1,44,토지등급,1980-05-13,수정
4,화성시 기산동 1-2,120,토지등급,1986-08-01,수정
...,...,...,...,...,...
195,화성시 기안동 19-11,43,토지등급,1980-05-13,수정
196,화성시 기안동 19-12,120,토지등급,1987-08-01,수정
197,화성시 기안동 19-12,117,토지등급,1986-08-01,수정
198,화성시 기안동 19-12,115,토지등급,1985-07-01,수정


In [32]:
print(len(landmark_history_address))
print(len(landmark_cate_lst))
print(len(landmark_area_lst))
print(len(landmark_move_rsn_lst))
print(len(landmark_move_ymd_lst))
print(len(landmark_related_address_lst))

36
37
36
36
36
36


csv 파일이 윈도우에서 오픈되지 않을 경우, 메모장으로 연 후 '모든파일에 대해 저장', 인코딩 = UTF-8(BOM)으로 설정하면 됩니다.